<a href="https://colab.research.google.com/github/Slebbon/TextGeneration_Projet_PSL_EnC/blob/main/FLAN_T2T.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **FLAN MODEL:** Text 2 Text

### **Importation des variables et libraires nécessaires**

In [1]:
! pip install -U accelerate
! pip install -U transformers
! pip install -U torch
! pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

### **Importation de l'ensemble des donnèes**

In [3]:
# Chargement du tokenizer et du modèle Flan pré-entraîné
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [5]:
from datasets import load_dataset
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

In [6]:
#Chargement des CSV
data_dir = r"C:\Users\marco\OneDrive\Desktop\TXTgeneration\Dataset\Divided_Dataset"

datasets = load_dataset('csv', data_files={
    'train': f'train.csv',
    'validation': f'validation.csv',
    'test': f'test.csv'
})

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [7]:
# Fonction de tokenisation des données
def tokenize_function(examples):
    model_inputs = tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=512)
    model_inputs["labels"] = tokenizer(examples["Author"], padding="max_length", truncation=True, max_length=512)["input_ids"]
    return model_inputs

# Tokeniser les ensembles de données
tokenized_datasets = datasets.map(tokenize_function, batched=True)

Map:   0%|          | 0/21054 [00:00<?, ? examples/s]

Map:   0%|          | 0/4513 [00:00<?, ? examples/s]

Map:   0%|          | 0/4512 [00:00<?, ? examples/s]

### **FLAN MODEL:** Entrainement

In [8]:
# Définir les paramètres d'entraînement
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [9]:
# Créer l'entraîneur
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer
)

In [10]:
# Effectuer le fine-tuning
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
path_to_save_fine_tuned_model = r"C:\Users\marco\OneDrive\Desktop\TXTgeneration"

In [ ]:
# Sauvegarder le modèle
model.save_pretrained(path_to_save_fine_tuned_model)
tokenizer.save_pretrained(path_to_save_fine_tuned_model)

### **FLAN:** Output

In [ ]:
# Chargement du modèle
model = AutoModelForSeq2SeqLM.from_pretrained(path_to_save_fine_tuned_model)
tokenizer = AutoTokenizer.from_pretrained(path_to_save_fine_tuned_model)

In [ ]:
# Fonction permettant de générer un texte dans le style souhaité
def generate_text(prompt, max_length=100):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
        inputs["input_ids"],
        max_length=max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        early_stopping=True
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
# Exemple d'utilisation
prompt_mixed = "Generate a speech in a style that combines Donald Trump and William Shakespeare:"
generated_text_mixed = generate_text(prompt_mixed)

print("Generated text in mixed style:")
print(generated_text_mixed)